In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_filer_date = dbutils.widgets.get("p_file_date")


#### Produce Constructor Standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
# race_result_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_filer_date}'") 

In [0]:
race_year_list = df_column_to_list(race_result_df,"race_year")

In [0]:
from pyspark.sql.functions import col

race_result_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))

In [0]:
display(race_result_df)

race_id,race_year,race_name,race_date,circuit_location,driver_name,driver_number,driver_nationality,team,grid,fastest_lap,race_time,points,position,file_date,created_date
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Mick Schumacher,null,German,Haas F1 Team,18,58,\N,0.0,16,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Nikita Mazepin,null,Russian,Haas F1 Team,19,55,\N,0.0,17,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Yuki Tsunoda,null,Japanese,AlphaTauri,20,49,+73.184,0.0,12,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Nicholas Latifi,6,Canadian,Williams,14,null,\N,0.0,null,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,George Russell,63,British,Williams,12,28,\N,0.0,null,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Lando Norris,4,British,McLaren,7,63,+23.702,15.0,3,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Charles Leclerc,16,Monegasque,Ferrari,4,60,+25.579,12.0,4,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Pierre Gasly,10,French,AlphaTauri,5,52,+52.818,6.0,7,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Antonio Giovinazzi,99,Italian,Alfa Romeo,17,57,\N,0.0,14,2021-04-18,2024-02-04T18:50:21.167Z
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00Z,Imola,Lance Stroll,18,Canadian,Aston Martin,10,59,+56.909,4.0,8,2021-04-18,2024-02-04T18:50:21.167Z


In [0]:
from pyspark.sql.functions import sum, count, when, col

constructor_standings_df = race_result_df \
    .groupBy("race_year","team") \
    .agg(sum("points").alias("total_points"),count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(constructor_standings_df.filter("race_year == 2020"))

race_year,team,total_points,wins


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))

final_df = constructor_standings_df.withColumn("rank",rank().over(constructor_rank_spec))

In [0]:
display(final_df.filter("race_year == 2020"))

race_year,team,total_points,wins,rank


In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")

# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")

# overwrite_function(final_df, 'f1_presentation', 'constructor_standings', 'race_year')

merge_condition = "tgt.team = src.team and tgt.race_year = src.race_year"
merge_delta_data(final_df, "f1_presentation", "constructor_standings", "race_year",presentation_folder_path, merge_condition)

In [0]:
%sql
select * from f1_presentation.constructor_standings where race_year = 2021;

race_year,team,total_points,wins,rank
2021,Mercedes,60.0,1,1
2021,Red Bull,53.0,1,2
2021,McLaren,41.0,0,3
2021,Ferrari,34.0,0,4
2021,AlphaTauri,8.0,0,5
2021,Aston Martin,5.0,0,6
2021,Alpine F1 Team,3.0,0,7
2021,Alfa Romeo,0.0,0,8
2021,Williams,0.0,0,8
2021,Haas F1 Team,0.0,0,8


In [0]:
%sql
select race_year,count(1) from f1_presentation.driver_standings
group by race_year
order by race_year desc;

race_year,count(1)
2021,20
2020,23
2019,20
2018,20
2017,25
2016,24
2015,22
2014,24
2013,23
2012,25
